# Density Consistency

In [1]:
using SparseArrays, LightGraphs
include("src/DC.jl")
using .DC

### Define model

In [2]:
# define graph
L = 4;
G = grid([L,L], periodic = true)
Adj = adjacency_matrix(G);
N = L^2

#set couplings
J = Adj.* rand([-1,1],N,N);

#set fields
h = fill(0.1, N);

### Create factor graph

In [5]:
β = .5;
ψ = DC.IsingFG(N,β*h,β*J, :EnergyFun); # computes moments numerically by running the trace over {-1,1}^2
#ψ = DC.IsingFG(N,β*h,β*J, :IsingPair); # computes moments analytically

### Run

In [14]:
ϕout , res, it, eps = density_consistency(ψ,N; epsconv = 1e-5, ρ = 0.95, closure = :DC, verbose = true, η = 1.0, convtype = :params);
#BP -> set η = 0.0 to compute BP fixed points (valid on any graph topology)
#ϕBP , res, it, epsv = density_consistency(ψ,N, epsconv = 1e-10, ρ = 0.9, closure = :DC, verbose = true, η = 0.);

Closure DC (η = 1.0) converged in 368 iterations w.r.t params - err params = 9.91673807559934e-6 - err moments = 1.0899944113962334e-7


In [15]:
ϕout.μt

16-element Array{Float64,1}:
 0.1035066003700775  
 0.10144281475811813 
 0.12836552774969454 
 0.09963165502711761 
 0.09243700705142507 
 0.005834533043558889
 0.0654609964588531  
 0.043284060076790525
 0.06518320794200853 
 0.017979686479076187
 0.08028922643916905 
 0.03763538901268927 
 0.019442835108429778
 0.0322079187665498  
 0.07550241214523841 
 0.012158877925237543

In [16]:
ϕout.Σtl #list of tilted covariance matrices, 1 for each factor in ψ

32-element Array{Array{T,2} where T,1}:
 [0.9892863996945103 0.2990145633297813; 0.2990145633297813 0.9897093608652913]    
 [0.9897093553339501 0.45165035417695176; 0.45165035417695176 0.9835222927972925]  
 [0.9892863803052502 0.5995392842113122; 0.5995392842113122 0.9900735122856658]    
 [0.9835223001501697 0.22628402446586393; 0.22628402446586393 0.9900735364060289]  
 [0.989286383679829 0.4262696640979723; 0.4262696640979723 0.9914553997273748]     
 [0.9897093486909211 -0.45462589621144295; -0.45462589621144295 0.9999659578685729]
 [0.9914554137357375 0.38856019851832346; 0.38856019851832346 0.999965957824319]   
 [0.9835222912855424 0.45374508433673433; 0.45374508433673433 0.995714857942614]   
 [0.9999659583667899 -0.48244183851623595; -0.48244183851623595 0.9957148562010866]
 [0.9900735333165575 0.5342055078030954; 0.5342055078030954 0.9981264501841786]    
 [0.991455412457729 0.517304540276573; 0.517304540276573 0.9981264638595067]       
 [0.9957148680113569 -0.513447037314

### Covariance matrix of tilted distribution
$$ \Sigma^{tilted}_{ij}\neq 0 \leftrightarrow (i,j)\in E $$

In [18]:
ϕout.Σt 

16×16 Array{Float64,2}:
  0.989286   0.299015  0.0       0.599539  …   0.0        0.0        0.0     
  0.299015   0.989709  0.45165   0.0           0.376417   0.0        0.0     
  0.0        0.45165   0.983522  0.226284      0.0        0.369305   0.0     
  0.599539   0.0       0.226284  0.990074      0.0        0.0        0.551409
  0.42627    0.0       0.0       0.0           0.0        0.0        0.0     
  0.0       -0.454626  0.0       0.0       …   0.0        0.0        0.0     
  0.0        0.0       0.453745  0.0           0.0        0.0        0.0     
  0.0        0.0       0.0       0.534206      0.0        0.0        0.0     
  0.0        0.0       0.0       0.0           0.0        0.0        0.0     
  0.0        0.0       0.0       0.0          -0.573071   0.0        0.0     
  0.0        0.0       0.0       0.0       …   0.0        0.601929   0.0     
  0.0        0.0       0.0       0.0           0.0        0.0       -0.64816 
 -0.439966   0.0       0.0       0.0    

### Covariance matrix of the full Gaussian distribution

In [20]:
ϕout.Σ

16×16 Array{Float64,2}:
  0.996419    0.301128    0.180147   …  -0.0117313  -0.124425    0.415002 
  0.301128    0.99656     0.455729       0.377848    0.0229258   0.0326176
  0.180147    0.455729    0.994483       0.0322067   0.372065   -0.0634056
  0.6037      0.201614    0.2283        -0.0123029  -0.170503    0.553272 
  0.429028   -0.0247019  -0.0645037     -0.0115049  -0.164789    0.260909 
  0.0778006  -0.456202   -0.298824   …  -0.280429   -0.107198    0.171008 
 -0.137081    0.258522    0.45692        0.0276746   0.386985   -0.318487 
  0.407946   -0.0256803  -0.111754      -0.0214744  -0.305054    0.491517 
 -0.137446    0.0972288   0.0240006      0.303418    0.0326582  -0.310656 
  0.0203805  -0.279843   -0.0260866     -0.573331    0.230977    0.0399313
 -0.167354    0.0224039   0.282537   …  -0.221426    0.604379   -0.404726 
 -0.329371    0.0231286   0.130095       0.0270519   0.39233    -0.648496 
 -0.441604    0.0340619  -0.0144919      0.360409    0.0926084  -0.517269 
 